<a href="https://colab.research.google.com/github/brs1977/BERT-Transformer-for-Summarization/blob/master/AbsBertSumPredict.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

#Setup

In [0]:
%cd /content
!git clone https://github.com/brs1977/BERT-Transformer-for-Summarization.git bertsum
%cd /content/bertsum

class ARGS(object):
    bert_model = 'bert-base-uncased'
    model_path =  'https://drive.google.com/uc?id=1-QRy87UMkoPCPVQrNbroC-XrIo1Qmj--&export=download'
    config_path = 'https://drive.google.com/uc?id=1-0QeYVALlC6iOlN8XEU6SyI6DUd50O5a&export=download'
    # batch_size = 16
    max_src_len = 130

args = ARGS()    

#download model and config
!gdown $args.model_path
!gdown $args.config_path


/content
fatal: destination path 'bertsum' already exists and is not an empty directory.
/content/bertsum
Downloading...
From: https://drive.google.com/uc?id=1-QRy87UMkoPCPVQrNbroC-XrIo1Qmj--
To: /content/bertsum/BertAbsSum_22.bin
816MB [00:04, 178MB/s]
Downloading...
From: https://drive.google.com/uc?id=1-0QeYVALlC6iOlN8XEU6SyI6DUd50O5a
To: /content/bertsum/config.json
100% 467/467 [00:00<00:00, 385kB/s]


#Model

In [0]:
import torch
from model import BertAbsSum
import json
from preprocess import convert_examples_to_features
from pytorch_pretrained_bert.tokenization import BertTokenizer


#create tokenizer
tokenizer = BertTokenizer.from_pretrained(args.bert_model)


device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
#load config
with open('/content/bertsum/config.json', 'r') as f:
    config = json.load(f)

#create model
model = BertAbsSum(args.bert_model, config['decoder_config'], device)
model.load_state_dict(torch.load('/content/bertsum/BertAbsSum_22.bin'))
model.to(device)


def convert_one_example(text, src_max_seq_length, tokenizer):
    src_tokens = tokenizer.tokenize(text)
    if len(src_tokens) > src_max_seq_length - 2:
        src_tokens = src_tokens[:(src_max_seq_length - 2)]
    src_tokens = ["[CLS]"] + src_tokens + ["[SEP]"]

    src_ids = tokenizer.convert_tokens_to_ids(src_tokens)

    src_mask = [1] * len(src_ids)
    src_padding = [0] * (src_max_seq_length - len(src_ids))
    src_ids += src_padding
    src_mask += src_padding

    return torch.tensor([src_ids]), torch.tensor([src_mask])    

def predict(predict_text, beam_size=5, n_best=5):
  """predict function"""
  src, src_mask = convert_one_example(predict_text, args.max_src_len, tokenizer)
  pred, _ = model.beam_decode(src.to(device), src_mask.to(device), beam_size=beam_size, n_best=n_best)  

  # De-tokenize.
  tok_text = " ".join(tokenizer.convert_ids_to_tokens(pred[0][0])).split('[SEP]')[0]
  tok_text = tok_text.replace(" ##", "")
  return tok_text.replace("##", "")  

#Predict

In [0]:

#predict
predict_text = '''Bidirectional Encoder Representations from Transformers (BERT) represents the latest incarnation of pretrained language models which have recently advanced a wide range of natural language processing tasks. In this paper, we showcase how BERT can be usefully applied in text summarization and propose a general framework for both extractive and abstractive models. We introduce a novel document-level encoder based on BERT which is able to express the semantics of a document and obtain representations for its sentences. Our extractive model is built on top of this encoder by stacking several inter-sentence Transformer layers. For abstractive summarization, we propose a new fine-tuning schedule which adopts different optimizers for the encoder and the decoder as a means of alleviating the mismatch between the two (the former is pretrained while the latter is not). We also demonstrate that a two-staged fine-tuning approach can further boost the quality of the generated summaries. Experiments on three datasets show that our model achieves state-of-the-art results across the board in both extractive and abstractive settings.'''
predict(predict_text)
